In [15]:
from facenet_pytorch import MTCNN, InceptionResnetV1, prewhiten
from facenet_pytorch.models.utils.detect_face import extract_face
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [17]:
def get_image(path, trans):
    img = Image.open(path)
    img = trans(img)
    return img

In [18]:
trans = transforms.Compose([
    transforms.Resize(512)
])

trans_cropped = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    prewhiten
])

In [19]:
dataset = datasets.ImageFolder('dataset/spoof_dataset', transform=trans)
dataset.idx_to_class = {k: v for v, k in dataset.class_to_idx.items()}
total_item = len(dataset)
# train_dataset, test_dataset = random_split(dataset, [total_item * .8, total_item * .2])
# train_loader, test_loader = DataLoader(train_dataset, collate_fn=lambda x: x[0]), DataLoader(test_dataset, collate_fn=lambda x: x[0])
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

In [20]:
mtcnn = MTCNN(device=device)

In [22]:
names = []
aligned = []
for img, idx in tqdm(loader):
    name = dataset.idx_to_class[idx]
    # start = time()
    img_align = mtcnn(img)#, save_path = "data/aligned/{}/{}.png".format(name, str(idx)))
    # print('MTCNN time: {:6f} seconds'.format(time() - start))

    if img_align is not None:
        names.append(name)
        aligned.append(img_align)

# aligned = torch.stack(aligned)

KeyboardInterrupt: 

In [ ]:
resnet = InceptionResnetV1(pretrained='casia-webface').eval().to(device)

In [ ]:
img = Image.open("dataset/emma1.jpg")
img_cropped1 = mtcnn(img)

In [ ]:
img_embedding1 = resnet(img_cropped1.unsqueeze(0).to(device)).cpu().detach().numpy()

# dist = np.linalg.norm(img_embedding1 - img_embedding2)
